In [1]:
import sys
sys.path.append("../")

import torch
import numpy as np
import torch_geometric as pyg

from models.Transformer_Block import Transformer_Block
from models.GD_Unroll import GD_Unroll
from models.RDPG_GD import RDPG_GD_Armijo, RDPG_GD_fixstep
from models.GD_Unroll_GCN import GCN


## Tiempos de inferencia

In [2]:
num_nodes = 500
n_components = 3
d = 10
device = 'cuda'
n = np.ones(n_components)*int(num_nodes/n_components)
n[n_components-1]=num_nodes-n[:n_components-1].sum()
p =  [[0.5, 0.1, 0.1],
      [0.1, 0.9, 0.1],
      [0.1, 0.1, 0.8]]

edge_index = pyg.utils.stochastic_blockmodel_graph(n, p).to(device)
edge_index_2 = torch.ones([num_nodes, num_nodes],).nonzero().t().contiguous().to(device)
ER_01 = pyg.utils.erdos_renyi_graph(num_nodes, 0.1, directed=False).to(device)
ER_03 = pyg.utils.erdos_renyi_graph(num_nodes, 0.3, directed=False).to(device)

x = torch.rand((1, num_nodes, d)).to(device)

In [3]:
lase = GD_Unroll(10, 10, 20, device=device)
lase.load_state_dict(torch.load('../saved_models/ugd_ini_fix_d10_full_2.pt'))
lase.to(device)
lase.eval()

GD_Unroll(
  (gd): GD_Block(
    (gcn): TAGConv(10, 10, K=1)
    (gat): Transformer_Block()
  )
)

In [4]:
lase_gcn = GD_Unroll(10, 10, 20, device='cuda')
lase_gcn.to(device)
lase_gcn.eval()

GD_Unroll(
  (gd): GD_Block(
    (gcn): TAGConv(10, 10, K=1)
    (gat): Transformer_Block()
  )
)

RDPG Gradient Descent with Armijo

In [5]:
%timeit -n 10 -r 10 RDPG_GD_Armijo(x.squeeze(0), edge_index)

4.34 ms ± 51.1 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


RDPG Gradient Descent with Armijo 20 iterations

In [6]:
%timeit -n 10 -r 10 RDPG_GD_Armijo(x.squeeze(0), edge_index, max_iter=20)

4.49 ms ± 403 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


LASE shared weights with 20 iterations

In [17]:
%timeit -n 10 -r 10 lase(x, edge_index, edge_index_2)

122 ms ± 1.11 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


LASE with only GCN with 20 layers (no attention)

In [12]:
%timeit -n 10 -r 10 lase_gcn(x, edge_index, edge_index_2)

37.9 ms ± 747 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [13]:
num_nodes = 1000
n_components = 3
d = 10
device = 'cuda'
n = np.ones(n_components)*int(num_nodes/n_components)
n[n_components-1]=num_nodes-n[:n_components-1].sum()
p =  [[0.5, 0.1, 0.1],
      [0.1, 0.9, 0.1],
      [0.1, 0.1, 0.8]]

edge_index = pyg.utils.stochastic_blockmodel_graph(n, p).to(device)
edge_index_2 = torch.ones([num_nodes, num_nodes],).nonzero().t().contiguous().to(device)
ER_01 = pyg.utils.erdos_renyi_graph(num_nodes, 0.1, directed=False).to(device)
ER_03 = pyg.utils.erdos_renyi_graph(num_nodes, 0.3, directed=False).to(device)

x = torch.rand((1, num_nodes, d)).to(device)

In [15]:
%timeit -n 10 -r 10 RDPG_GD_Armijo(x.squeeze(0), edge_index)

8.86 ms ± 165 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [16]:
%timeit -n 10 -r 10 RDPG_GD_Armijo(x.squeeze(0), edge_index, max_iter=20)

8.88 ms ± 168 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [18]:
%timeit -n 10 -r 10 lase(x, edge_index, edge_index_2)

122 ms ± 703 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [5]:
num_nodes = 5000
n_components = 3
d = 10
device = 'cuda'
n = np.ones(n_components)*int(num_nodes/n_components)
n[n_components-1]=num_nodes-n[:n_components-1].sum()
p =  [[0.5, 0.1, 0.1],
      [0.1, 0.9, 0.1],
      [0.1, 0.1, 0.8]]

edge_index = pyg.utils.stochastic_blockmodel_graph(n, p).to(device)
edge_index_2 = torch.ones([num_nodes, num_nodes],).nonzero().t().contiguous().to(device)
ER_01 = pyg.utils.erdos_renyi_graph(num_nodes, 0.1, directed=False).to(device)
ER_03 = pyg.utils.erdos_renyi_graph(num_nodes, 0.3, directed=False).to(device)

x = torch.rand((1, num_nodes, d)).to(device)

In [20]:
%timeit -n 10 -r 10 RDPG_GD_Armijo(x.squeeze(0), edge_index)

135 ms ± 39.7 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [21]:
%timeit -n 10 -r 10 RDPG_GD_Armijo(x.squeeze(0), edge_index, max_iter=20)

135 ms ± 226 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [6]:
%timeit -n 10 -r 10 lase(x, edge_index, edge_index_2)

OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB (GPU 0; 11.75 GiB total capacity; 10.62 GiB already allocated; 138.06 MiB free; 10.74 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF